at first make sure to run app.py. In this regard you should go to the app.py repository and run "flask run" command in your terminal. If you install all the requirment correctly, you should see in your command line that the flask is running in this server: Running on http://127.0.0.1:5000
The server and the port should be the same otherwise you should change the code below.

In [262]:
from serving_client import ServingClient


In [263]:
from game_client import GameClient

In [264]:
gc = GameClient()

In [265]:
#get data

In [266]:
df = gc.ping_game(2022020323)

In [267]:
#predict

In [268]:
df.head()

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
0,23,02:38,1,2022020323,Ottawa Senators,Anaheim Ducks,0,64.0,-29.0,1,...,5,4,89,38.2884,67.268120,0.8593,1.2572,38,0,0
1,24,02:57,1,2022020323,Ottawa Senators,Anaheim Ducks,0,72.0,-7.0,5,...,5,4,108,18.3848,23.409400,0.3906,0.8593,57,0,0
2,25,03:11,1,2022020323,Ottawa Senators,Anaheim Ducks,0,39.0,31.0,5,...,5,5,0,58.8303,50.328918,0.5550,0.3906,71,0,0
3,26,03:39,1,2022020323,Ottawa Senators,Anaheim Ducks,0,46.0,28.0,5,...,5,5,0,51.3128,7.615773,0.5772,0.5550,39,0,0
4,44,06:45,1,2022020323,Ottawa Senators,Anaheim Ducks,0,-66.0,-30.0,2,...,5,5,0,37.8021,135.236829,0.9167,0.8330,45,0,0


In [269]:
# from serving_clinet import ServingClient

In [270]:
sc= ServingClient()

In [271]:
sc.download_registry_model(workspace='ift-6758-2', model='xgb-model-5-2-pickle', version='1.0.0', model_name='model_5_2.pickle')


'model_5_2.pickle'

In [272]:
d = sc.predict(df)

In [273]:
print(d)

[0.11344169080257416, 0.11385823041200638, 0.01134471595287323, 0.014089926145970821, 0.03645966202020645, 0.08605697751045227, 0.13325197994709015, 0.01674901507794857, 0.011195152066648006, 0.0445728674530983, 0.106532983481884, 0.14451998472213745, 0.03590341657400131, 0.12089139968156815, 0.04576929286122322, 0.16789552569389343, 0.020967621356248856, 0.01727023907005787, 0.16990028321743011, 0.09372556209564209, 0.2586909532546997, 0.1258142739534378, 0.09846320003271103, 0.16328299045562744, 0.015934152528643608, 0.028903089463710785, 0.10588010400533676, 0.15201084315776825, 0.07989291846752167, 0.14780405163764954, 0.11236900836229324, 0.03289693966507912, 0.21448811888694763, 0.16163036227226257, 0.1199759915471077, 0.008039368316531181, 0.08849029242992401, 0.12147155404090881, 0.15845122933387756, 0.043525826185941696, 0.03949134424328804, 0.04901617020368576, 0.017767254263162613, 0.05603015422821045, 0.06276930123567581, 0.07922637462615967, 0.1918400228023529, 0.188170418